In [ ]:
import tensorflow as tf
from tensorflow import keras
from glob import glob
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras import datasets, layers, models

drive.mount('/content/gdrive')

In [ ]:
label_list = ['cat', 'dog', 'dino', 'fox', 'rabbit']
label_dic = {'cat':0, 'dog':1, 'dino':2, 'fox':3, 'rabbit':4}
N_CLASS = 5
N_DATA = 0
N_EPOCHS = 5
N_BATCH = 20
learning_rate = 0.01
count_dic = {}
labelandImage = []

dir_name = "/content/gdrive/MyDrive/faceImages/"
for label in label_list:
  filenames = glob(dir_name+label+ "/*.jpg")
  for filename in filenames:
    labelandImage.append((label, filename))

for label, imagePath in labelandImage:
  try: count_dic[label] += 1
  except: count_dic[label] = 1
N_DATA = sum(count_dic.values())

print(count_dic)
print(f"Total : {N_DATA}")

In [ ]:
X, Y = [], []
for label, imagePath in labelandImage:
  image = Image.open(imagePath).resize((128, 128)).convert("L")
  image = np.array(image)
  X.append(image)
  Y.append(label_dic[label])
X, Y = np.array(X), np.array(Y)

train_images, test_images, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=44)

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.


N_TRAIN = train_images.shape[0]
N_TEST = test_images.shape[0]
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=N_TRAIN).batch(N_BATCH).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(N_BATCH)

print(f"train image shape: {train_images.shape}")
print(f"test image shape: {test_images.shape}")

In [ ]:
# sequential API를 활용하여 model 구성
def create_model():
  model = tf.keras.Sequential()
  model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='SAME', input_shape=(128, 128, 1)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(256, activation='relu'))
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Dense(N_CLASS, activation='softmax'))
  return model
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
steps_per_epoch = N_TRAIN/N_BATCH
validation_steps = N_TEST/N_BATCH

history = model.fit(train_dataset, epochs=N_EPOCHS, steps_per_epoch=steps_per_epoch, validation_data=test_dataset, validation_steps=validation_steps)

In [ ]:
model.evaluate(test_dataset)

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], 'b-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'r--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img)

  predicted_label = np.argmax(predictions_array)
  color = ''
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(label_list[predicted_label], 100*np.max(predictions_array), label_list[true_label]), color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks(range(N_CLASS), label_list, rotation=90)
  plt.yticks([])

  thisplot = plt.bar(range(N_CLASS), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
rnd_idx = np.random.randint(1, N_TEST//N_BATCH)
img_cnt = 0
for images, labels in test_dataset:
  img_cnt += 1
  if img_cnt != rnd_idx: continue

  predictions = model(images, training=False)
  num_rows, num_cols = 2, 4
  num_images = num_rows * num_cols
  labels = tf.argmax(labels, axis=1)
  plt.figure(figsize=(3*2*num_cols, 4*num_rows))
  plt.subplots_adjust(hspace=1.0)
  
  for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions.numpy(), labels.numpy(), images.numpy())
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions.numpy(), labels.numpy())
  break

In [ ]:
# model.save('/content/gdrive/MyDrive/animallike.h5')